<a href="https://colab.research.google.com/github/juan-orea/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/Colonia_de_Hormigas_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tsplib95

import urllib.request
import tsplib95
import random
from math import e
from math import pow

In [0]:
file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f3d967682b0>)

In [0]:
problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges()) 

In [0]:
# Funciones auxiliares: distancia
#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

In [0]:
# Funciones auxiliares
def NivelFeromona(v):
  return (v) / 10 if v <= 3.5 else 1 if v >= 1 else 0.1

def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas

  # alfa = 1.5
  # beta = 2.5
  nodos = list(problem.get_nodes())
  descartados = set(H)
  # print(H)
  nodoActual = H[len(H) - 1]
  Pesos = [ 0 if nodos[i] in descartados else 
           math.pow(NivelFeromona(T[nodoActual][nodos[i]]), alfa) * math.pow(50 / distancia(nodoActual, nodos[i], problem),beta)
           for i in range(len(nodos)) ]
  # print(Pesos)
  # print(H)
  # if len(descartados) == 2:
  #  print(Pesos)
  return random.choices(nodos, weights = Pesos)[0]


def Incrementa_Feromona(problem, T, H ) :
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 2000/distancia_total(H, problem)
  return T


def Evaporar_Feromonas(T ):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T


In [0]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges())
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      # print(Hormiga)
      # print(Hormiga[h])
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      # motivación extra
      # T = Incrementa_Feromona(problem, T, Hormiga[h] )
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
      print('Paso {0}, distancia = {1}: {2}'.format(h, mejor_distancia, mejor_solucion))
  
  
  # dibujarGrafo(mejor_solucion, problem)
  print(mejor_solucion)
  print(mejor_distancia)


In [282]:
# estas variables tendrían que ser locales a la funcion Add_Nodo
alfa = 1.3  # 1.3
beta = 3.5  # 3.5
hormigas(problem, 2000)

Paso 0, distancia = 2135: [0, 1, 7, 5, 6, 4, 27, 2, 32, 10, 8, 41, 23, 9, 21, 39, 22, 38, 3, 28, 30, 29, 34, 20, 33, 36, 35, 17, 37, 14, 15, 16, 19, 13, 25, 12, 11, 18, 26, 31, 40, 24]
Paso 2, distancia = 1752: [0, 1, 37, 14, 16, 15, 7, 6, 4, 3, 2, 27, 32, 34, 20, 33, 38, 22, 39, 21, 40, 24, 8, 9, 23, 41, 25, 12, 11, 18, 5, 19, 13, 26, 29, 30, 28, 10, 17, 31, 35, 36]
Paso 23, distancia = 1691: [0, 1, 27, 2, 3, 4, 6, 5, 26, 18, 11, 12, 25, 10, 8, 9, 23, 41, 29, 30, 38, 22, 39, 21, 40, 24, 34, 33, 20, 32, 17, 37, 15, 16, 14, 19, 13, 28, 7, 31, 35, 36]
Paso 25, distancia = 1621: [0, 1, 3, 27, 2, 30, 29, 28, 4, 6, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 40, 24, 21, 39, 38, 22, 32, 34, 33, 20, 7, 31, 17, 37, 15, 16, 14, 19, 13, 35, 36]
Paso 29, distancia = 1608: [0, 1, 3, 4, 27, 2, 28, 30, 29, 23, 41, 25, 10, 8, 9, 40, 24, 21, 39, 22, 38, 32, 20, 34, 33, 37, 15, 16, 14, 19, 13, 26, 18, 11, 12, 5, 6, 7, 31, 17, 35, 36]
Paso 44, distancia = 1562: [0, 1, 6, 4, 3, 2, 27, 28, 30, 29, 9, 8, 10, 

In [0]:
print("Nodos", Nodos)
print("Aristas", Aristas)


In [0]:
import matplotlib.pyplot as plt
import networkx as nx

def dibujarGrafo(solucion, problema):
    
    # Declaración de Gráfico a utilizar: grafo dirigido
    G=nx.DiGraph()
    
    for n in solucion:
      # Añadimos los nodos
      G.add_node(str(n))

    # la intencion es dar la referencia de distancias entre todos los nodos, para que los posicione sobre el plano (pero no funciona así)
    distancias = { u:{ v: distancia(u, v, problema) for v in range(len(solucion)) if u != v } for u in range(len(solucion)) }

    # Este paso es necesario para definir la colocación de los nodos y aristas a dibujar
    # Básicamente, este método genera una distribución ajustada al modelo de los nodos y aristas en el espacio
    # Existen varias maneras de generar esta distribución pero la que he encontrado que mejor se ajusta es ésta
    # ya que esta función utiliza el atributo 'weight' de las aristas para ajustar la posición de los nodos
    # Otros: https://networkx.github.io/documentation/latest/reference/drawing.html#layout
    # Además existen varias librerías para generar vistas con distintas distribuciones
    pos=nx.kamada_kawai_layout(G, dist=distancias, weight=None)
    
    nodo_previo = None
    for n in solucion:
      if nodo_previo != None:
        # Añadimos los caminos elegidos (para ir desde nodo_previo al nodo n)
        G.add_edge(str(nodo_previo),str(n), weight=distancia(nodo_previo, n, problema))
      nodo_previo = n
      
    # falta la arista de vuelta al inicio
    G.add_edge(str(nodo_previo), str(solucion[0]), weight=distancia(nodo_previo, n, problema))

    # Este paso es necesario si queremos mostrar el valor de las aristas
    # Generamos un diccionario de las aristas para poder dibujarlas en el grafo a continuación
    # Utilizamos el atributo 'weight' de las aristas para asignar un valor a la etiqueta
    edge_labels=dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])
    

    
    # Se dibuja el grafo con las etiquetas de los nodos utilizando las posiciones generadas anteriormente
    nx.draw(G,pos,with_labels=True)
    
    # Se dibujan las aristas direccionadas entre los nodos conectados con sus valores
    nx.draw_networkx_edge_labels(G,pos,with_labels=True,edge_labels=edge_labels)
